# Interactive Functions 

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display


from shared_utils import geography_utils, styleguide
from shared_utils import calitp_color_palette as cp 


/Users/andrewvaccaro/go/src/github.com/cal-itp/data-analyses/.venv/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
import _dla_utils

In [3]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [4]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [5]:
df.sample(5)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
7833,Obligated,RPSTPL,5240(029),Culver City,2015-12-01,2015-12-01,2015-12-01,2015-12-01,2015-12-18,-14426.14,0.00,19576.97,Authorized,5240,7,E-76 approved on Prepared on 10/6/2015,0.00,0.00,17.00,LAF1659,Construction Of Safety And Aesthetic Relpedest...,"Ada Ramp, Crosswalk Pavement Treatment, Markin...",6,NaT,NaT,SCAG,5240,2015.00,Culver City,22381.40,-16492.70,0.00,1,0,0,1,0,0,0,2
19036,Obligated,BRLS,5907(067),Lassen County,2021-04-07,2021-04-08,2021-04-22,2021-05-03,2021-05-19,0.00,0.00,0.00,Authorized,5907,2,E-76 approved on,1.00,25.00,16.00,HBPLOCAL,Lassen Creek And Sand Slough Bridges On Johnst...,Replace Bridge Rail Using Bridge Preventive Ma...,2,2021-04-07,2021-04-07,NON-MPO,5907,2021.00,Lassen County,0.00,0.00,0.00,0,1,1,0,0,1,0,3
20139,Obligated,HSIPL,5484(013),Wildomar,2021-10-07,2021-10-07,2021-10-07,2021-10-07,2021-10-13,-46845.74,0.00,-52050.60,Authorized,5484,8,E-76 approved on,7.00,0.00,6.00,SCAG015,Various Locations Throughout The City Of Wildomar,Upgrade Existing Guardrail And Associated Impr...,2,2021-09-30,2021-10-01,SCAG,5484,2021.00,Wildomar,-52050.64,-46845.78,0.00,0,1,0,1,0,1,0,3
9859,Obligated,BRLS,5908(070),Tehama County,2015-01-13,2015-02-13,2015-02-17,2015-02-18,2015-02-18,28330.00,0.00,32000.00,Authorized,5908,2,E-76 approved on E-76 approved on,31.00,5.00,0.00,HBPLOCAL,Hwy 99 At Thomes Creek (br.# 08c0110),Bridge Replacement,6,NaT,NaT,NON-MPO,5908,2015.00,Tehama County,36584.04,32388.31,0.00,0,0,1,0,0,1,0,2
20501,Obligated,HSIPL,5071(019),South Pasadena,2021-11-30,2021-11-30,2021-12-06,2021-12-06,2021-12-09,0.00,0.00,0.00,Authorized,5071,7,E-76 approved on,8.00,6.00,3.00,SCAG015,"Mission St/diamond Ave, Mission St/fairview Av...",Upgrade Crosswalks; Install In-roadway Warning...,5,2021-11-22,2021-11-30,SCAG,5071,2021.00,South Pasadena,0.00,0.00,0.00,1,0,0,1,0,0,0,2


## Full Function for Prefix

In [6]:
    @interact
    
    def dla_get_prefix(place=df.primary_agency_name.sort_values().unique().tolist()):
        
        agencies = df[df.primary_agency_name==place]
    
        prefix_count_n = agencies >> count(_.prefix)
    
        display(Markdown(f"**Summary Statistics for {place}**"))
        display(Markdown(f"The number of obligations {place} has is {len(agencies)}"))
        
        display(Markdown(f"The number of prefix codes {place} uses is {len(prefix_count_n)}"))
        
        
        
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        display(df[df.agency == place][['fed_requested','ac_requested','total_requested']].describe())
        
        display(Markdown(f"**Top Project Types in {place}**"))
        display((df[df.primary_agency_name == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
         

    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        
        ax1 = (styleguide.preset_plotnine_config(ax1)
         + scale_fill_manual(values=cp.CALITP_CATEGORY_BOLD_COLORS)
        )
        
        return ax1



interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Alameda', 'Al…

## Function for results by Prefix

In [12]:
@interact
def prefix_all_agencies_4(prefix_unique=df.prefix.sort_values().unique().tolist()):
    
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n))
    
    prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix_unique', options=('6140000', 'ACNHPI', 'ACSTER', 'ACSTP', '…

## Function for Type of Work

In [8]:
top50 = (df >> count(_.type_of_work) >> arrange(-_.n)).head(50)

In [9]:
@interact
def findtypeofwork(work=top50.type_of_work.sort_values().unique().tolist()):
    
    display(Markdown(f"**Number of Obligations the top Agencies have in {work}**"))
    # graphs 
    worktypes = df[df.type_of_work== work]
    
    work_count = (worktypes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    display(work_count)

    ax1 = (work_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Agencies using Type of Work', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1

interactive(children=(Dropdown(description='work', options=('Ac Overlay', 'Ac Overlay (tc)', 'Asphalt Concrete…

## Function to Filter Data

In [10]:
df_query = df[['prefix', 'primary_agency_name', 'project_no','locode','dist','fed_requested', 'ac_requested', 'total_requested',
              'type_of_work','project_location', 'seq', 'mpo','prepared_y']]

In [11]:
#adding year
@interact
def get_query2(agency=df_query.primary_agency_name.unique().tolist(),
             prefix=df_query.prefix.unique().tolist(),
             year=df_query.prepared_y.unique().tolist()):
   
    filtering = df_query[(df_query.primary_agency_name==agency)&(df_query.prefix==prefix)&(df_query.prepared_y==year)]
    
    
    display(Markdown(f"**The number of obligations {agency} has in {prefix} during the year {year} is {len(filtering)}**"))
    
    display(filtering.head())
    
    

interactive(children=(Dropdown(description='agency', options=('Humboldt County', 'Mendocino County', 'Sacramen…